# Step 1: Build ASR pipeline
## Data preprocessing

In [1]:
from data_preparation.wnut import wnut_to_json

wnut_to_json("datasets/wnut/wnut17train.conll", 
                  "datasets/wnut/emerging.dev.conll",
                  "datasets/wnut/emerging.test.conll")

Annotation error:  datasets/wnut/emerging.test.conll 18828 ['Advertise', 'I-creative-work']
Annotation error:  datasets/wnut/emerging.test.conll 18829 ['Anything', 'I-creative-work']


In [2]:
types['entities']

{'location': {'short': 'location'},
 'group': {'short': 'group'},
 'corporation': {'short': 'corporation'},
 'person': {'short': 'person'},
 'creative-work': {'short': 'creative-work'},
 'product': {'short': 'product'}}

In [7]:
test[204]

{'tokens': ['For',
  'more',
  'info',
  'about',
  'this',
  'and',
  'local',
  'views',
  'on',
  'the',
  'matter',
  'check',
  'out',
  'where',
  'OP',
  'took',
  'this',
  'from',
  '.'],
 'extended': ['For',
  'more',
  'info',
  'about',
  'this',
  'and',
  'local',
  'views',
  'on',
  'the',
  'matter',
  'check',
  'out',
  'where',
  'OP',
  'took',
  'this',
  'from',
  '.'],
 'entities': [Entity(type='creative-work', start=6, end=8)]}

In [4]:
from data_preprocessing.tokenize import tokenize_json
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small", model_max_length=4096)

tokenize_json(tokenizer,
              "datasets/wnut/wnut17train.json",
              "datasets/wnut/emerging.dev.json",
              "datasets/wnut/emerging.test.json",
              "datasets/wnut/wnut_types.json")

/Users/robinloebbert/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# labels can have multiple values concatenated via commas!

In [1]:
from data_preprocessing.tensorize import NERDataProcessor
from data_preprocessing.tokenize import MENTION_START, MENTION_END
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small", model_max_length=4096)
tokenizer.add_tokens(MENTION_START)
tokenizer.add_tokens(MENTION_END)

config = {
    "mention_start_token": MENTION_START,
    "mention_end_token": MENTION_END
}

processor = NERDataProcessor(config, tokenizer,
                             "datasets/wnut/wnut17train.t5-small.jsonlines",
                             "datasets/wnut/emerging.dev.t5-small.jsonlines",
                             "datasets/wnut/emerging.test.t5-small.jsonlines",
                             "datasets/wnut/wnut_types.json")

/home/loebbert/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
processor.get_cache_path()

'datasets/wnut/cached.tensors.t5-small.bin'

In [2]:
train, dev, test = processor.get_tensor_samples()

In [3]:
processor.stored_info["example"]["emerging.test_22"]

{'doc_id': 'emerging.test_22',
 'sentence': ['▁Rep',
  'ly',
  'ing',
  '▁to',
  '▁another',
  '▁question',
  ',',
  '▁B',
  'ham',
  're',
  '▁said',
  '▁the',
  '▁jaw',
  'ans',
  '▁deployed',
  '▁at',
  '▁places',
  '▁such',
  '▁as',
  '▁Si',
  'a',
  'chen',
  '▁Gla',
  'cier',
  '▁are',
  '▁provided',
  '▁with',
  '▁the',
  '▁best',
  '-',
  'quality',
  '▁winter',
  '▁clothing',
  '.',
  '</s>'],
 'input_sentence': ['▁named',
  '▁entity',
  '▁recognition',
  ':',
  '▁Rep',
  'ly',
  'ing',
  '▁to',
  '▁another',
  '▁question',
  ',',
  '▁B',
  'ham',
  're',
  '▁said',
  '▁the',
  '▁jaw',
  'ans',
  '▁deployed',
  '▁at',
  '▁places',
  '▁such',
  '▁as',
  '▁Si',
  'a',
  'chen',
  '▁Gla',
  'cier',
  '▁are',
  '▁provided',
  '▁with',
  '▁the',
  '▁best',
  '-',
  'quality',
  '▁winter',
  '▁clothing',
  '.',
  '</s>'],
 'target_sentence': ['▁Rep',
  'ly',
  'ing',
  '▁to',
  '▁another',
  '▁question',
  ',',
  '<m>',
  '<m>',
  '▁B',
  'ham',
  're',
  '</m>',
  '</m>',
  '▁said'

In [4]:
import pickle

test_multiple = None
for (doc_key, subtoken_map, sample) in test.data:
    if doc_key == "emerging.test_22":
        test_multiple = (doc_key, subtoken_map, sample)
        break
with open("tests/data/wnut_nested_batch_1.pkl", "wb") as file:
    pickle.dump(test_multiple, file)

with open("tests/data/wnut_nested_batch_10.pkl", "wb") as file:
    res = [test_multiple, *test.data[:9]]
    print(len(res))
    pickle.dump(res, file)

with open("tests/data/wnut_batch_1.pkl", "wb") as file:
    pickle.dump(train.data[0], file)

with open("tests/data/wnut_batch_10.pkl", "wb") as file:
    pickle.dump(train.data[:10], file)

10


In [2]:
5/3, 5//3

(1.6666666666666667, 1)

In [5]:
from data_preprocessing.tensorize import ner_collate_fn
import pickle

with open("tests/data/wnut_batch_10.pkl", "rb") as file:
            data_point = pickle.load(file)
(doc_keys, subtoken_maps, batch) = ner_collate_fn(data_point)
assert len(subtoken_maps) == 10

In [6]:
subtoken_maps

([0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  5,
  6,
  6,
  6,
  6,
  6,
  6,
  7,
  8,
  8,
  9,
  9,
  9,
  10,
  11,
  12,
  13,
  13,
  13,
  13,
  13,
  14,
  14,
  15,
  16,
  17,
  18,
  18,
  18,
  18,
  18,
  19,
  19,
  19,
  19,
  19,
  19,
  20,
  20,
  20,
  20,
  21,
  22,
  22,
  22,
  22,
  23,
  23,
  23,
  23,
  23,
  24,
  24,
  24,
  24,
  25,
  26,
  26,
  27,
  28,
  29,
  30],
 [0,
  0,
  1,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  5,
  6,
  6,
  7,
  7,
  7,
  8,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  10,
  11,
  12,
  12,
  13,
  13,
  14,
  14,
  14,
  15,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  16,
  17,
  18,
  19,
  19,
  20,
  21,
  22,
  22,
  22,
  23,
  24,
  24,
  25,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  31,
  32,
  32,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  33,
  34,
  34,
  34,
  34,
  34,
  35],
 [0,
  0,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  3,
  4,
  5,

In [6]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")

input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# forward pass
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
last_hidden_states = outputs.last_hidden_state

/Users/robinloebbert/opt/miniconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [1]:
import torch

torch.__version__

'2.0.0'

In [9]:
import os
"/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])

'/home/loebbert/projects'

In [1]:
from models.metrics import F1ASP

metric = F1ASP()

/home/loebbert/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
metric.update([(2, 3, 5)], [])
metric.update([], [])

In [28]:
metric.compute()

0.6530612707138062